<h3 align ="center">Social Network Analysis: Machine Learning for Github User’s Recommendations System</h3>

<h5> Institution: </h5>
<ul> 
   <li> National High School Of Computer Science and System Analysis </li>
    <li> Mohammed V University </li>
</ul>

<h5> Course: </h5>
<ul> 
   <li> Master in Data Science and Big Data </li>
   <li> Social Network Analysis Module
   <li> 3rd Semester </li>
</ul>

<h5> Authors: </h5>
<ul> 
   <li > Rida Boushab</li>
    <li > Vasconselos Fernando Victor Oliveira</li>
</ul>
 

In [1]:
import github_api_v3 as gt
import os
import requests
import numpy as np
import pandas as pd
import networkx as nx
from neo4j import GraphDatabase, basic_auth
from py2neo import Graph, Node, Relationship 

In [2]:
#url = "https://api.github.com/repositories"
#github_api_v3.getall(url, headers= None)
username = 'VittorOlliveira'
token = 'debe77cbda5a5e8284bc2232315582dde8f1a1b0'

In [3]:

url = {}
star_url = {}
languages_url = {}
commits_url = {}
issues_url = {}
commits = {}
repos_languages = {}
languages_repos = {}
repos_readme = {}
commiter = {}
repos_name = set()
repos_stargazers = {}
commit_list= []
issue_assignee = {}
issue_creator = {}
labels = {}
repo_labels = {}


In [4]:
#for i in range(len(url)):
#r1.json()
r = requests.get('https://api.github.com/repositories', auth=(username,token))

for entry in r.json():
    url[entry['name']]= 'https://api.github.com/repos/' + entry['owner']['login'] + '/' + entry['name'] + '/readme'
    star_url[entry['name']]= 'https://api.github.com/repos/' + entry['owner']['login'] + '/' + entry['name'] + '/stargazers'
    languages_url[entry['name']]= 'https://api.github.com/repos/' + entry['owner']['login'] + '/' + entry['name'] + '/languages'
    commits_url[entry['name']]= 'https://api.github.com/repos/' + entry['owner']['login'] + '/' + entry['name'] + '/commits'
    issues_url[entry['name']]= 'https://api.github.com/repos/'+ entry['owner']['login']+'/'+ entry['name']+'/issues'
    labels[entry['name']]= 'https://api.github.com/repos/'+ entry['owner']['login']+'/'+ entry['name']+'/labels'
for x in url:
    repos_readme[x]= requests.get(url[x], auth=(username, token))

x2remove = []
for x in repos_readme:
    if ("download_url" in repos_readme[x].json()) == False:
        x2remove.append(x) 
        
for i in range(len(x2remove)):
    repos_readme.pop(x2remove[i])

for x in repos_readme:
    #requests.get(x.json()['download_url'], allow_redirects=True)
    name = '' + x +'.md'
    open(name, 'wb').write(requests.get(repos_readme[x].json()['download_url'], allow_redirects=True).content)

In [25]:
for x in star_url:
    if x in repos_readme: 
        repos_stargazers[x]= requests.get(star_url[x], headers={"Accept":"application/vnd.github.v3.star+json"}, auth=(username, token))

repos_stargazers_login = {}
list_star = []
list_starred_time = []
for x in repos_stargazers:
    list_star = []
    y=repos_stargazers[x].json()
    for i in range(len(y)):
        list_star.append(y[i]['user']['login'])
        list_starred_time.append(y[i]['starred_at'])
    repos_stargazers_login[x] = [list_star,list_starred_time] 

#for x in repos_stargazers_login:
#    print(repos_stargazers_login[x])

In [6]:
for x in languages_url:
    if x in repos_readme: 
        repos_languages[x]= requests.get(languages_url[x], auth=(username, token))
        
#for x in repos_languages:
#    for y in repos_languages[x].json():
#        languages_repos[x] = y
#        print(y)

In [7]:
for x in commits_url:
    if x in repos_readme: 
        commits[x]= requests.get(commits_url[x], auth=(username, token))
        
for x in commits:
    commit_list= []
    for i in range(len(commits[x].json())):
         commit_list.append(commits[x].json()[i]['committer'])  
    commiter[x] = commit_list
    
#for x in commiter:
#    print(commiter[x])
#    for i in range(len(commiter[x])):
#         if type(commiter[x][i]) != type(None):
#            print(x, commiter[x][i]['login'])

In [8]:
for x in issues_url:
    if x in repos_readme: 
        issue_assignee[x] = requests.get(issues_url[x], auth=(username, token))

In [9]:
issue_dict = {} 
for x in issue_assignee:
    issue = []
    issue_time = []
    for i in range(len(issue_assignee[x].json())):
        if type(issue_assignee[x].json()[i]['user']) is not type(None):
           # print(issue_assignee[x].json()[i]['user']['login'])
           # issue.append(issue_assignee[x].json()[i]['user']['login'])
           # print(issue_assignee[x].json()[i]['created_at'])
           # issue_time.append(issue_assignee[x].json()[i]['created_at'])
            issue.append({"assignee":issue_assignee[x].json()[i]['user']['login'], "time":issue_assignee[x].json()[i]['created_at']})
    issue_creator[x]= issue
#for x in issue_creator:
   # print("List of assignee of issues of the repos "+ x +": ", issue_creator[x])

Get labels

In [10]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
graph = Graph("bolt://localhost:7687")
#a = Node("Person", name="Alice")
#b = Node("Person", name="Bob")
#ab = Relationship(a, "KNOWS", b)
#graph.create(ab)



In [11]:
for x in r.json():
    names = x['name']
    
    if x['name'] in repos_readme:
        owner =  x['owner']['login']
        readme ='' + names +'_readme.md'
        
        for l in repos_languages[names].json():
            language = l
            
        if owner is not owner.replace('-','_'):
            owner = owner.replace('-','_')
                
        if names is not names.replace('-','_'):
            names = names.replace('-','_')
            
        user = Node("u:user", name=""+owner)
        repos = Node("r:repository", name= ""+names, read_me= ""+readme, lang=language )
        ur = Relationship(user,"OWNS", repos)
        if type(graph.nodes.match("u:user", name=""+owner).first()) is not type(None):
            user = (graph.nodes.match("u:user", name=""+owner).first())
            if type(graph.nodes.match("r:repository", name=""+names).first()) is not type(None):
                repos = (graph.nodes.match("r:repository", name=""+names).first())
                ur = Relationship(user,"OWNS", repos)
                graph.merge(ur)
            else:
                graph.create(repos)
                repos=(graph.nodes.match("r:repository", name=""+names).first())
                ur = Relationship(user,"OWNS", repos)
                graph.merge(ur)
        else: 
            graph.create(ur)
            
            #cqlNodeQuery = """
            #CREATE ("""+owner+""":user {name: '""" + owner + """'})-[o:owns]->(r:repository {name:' """ + name + """ ', 
            #readme: '"""+ readme + """', 
            #languages: '""" +language+"""'  
            #})
            #"""
            #nodes = graphDB_Session.run(cqlNodeQuery)

In [12]:
u = (graph.nodes.match("u:user", name=""+owner).first())
u


(_66:`u:user` {name: 'collectiveidea'})

In [13]:
with driver.session() as graphDB_Session:
    commiters =[]
    for x in r.json():
        name = x['name']
        commiters =[]
        name = name.replace('-','_')
        if name in commiter:
            for i in range(len(commiter[name])):
                if type(commiter[name][i]) is not type(None): 
                    commiters.append(commiter[name][i]['login'])
                    if commiter[name][i]['login'] is not commiter[name][i]['login'].replace('-','_'):
                        commiter[name][i]['login'] = commiter[name][i]['login'].replace('-','_')
                    
                    if commiter[name][i]['login'] is not commiter[name][i]['login'].replace('7',''):
                        commiter[name][i]['login'] = commiter[name][i]['login'].replace('7','')
                    
                    user = (graph.nodes.match("u:user", name=""+commiter[name][i]['login']).first())
                    repos = (graph.nodes.match("r:repository", name=""+name).first())
                    if type(user) is not type(None):
                        ur = Relationship(user, "COMMITS", repos)
                        graph.merge(ur)
                   #cqlNodeQuery = """MERGE ("""+commiter[name][i]['login']+""":user {name: '""" + commiter[name][i]['login'] + """'})
                   # ON CREATE SET """+ commiter[name][i]['login']+""".name = '"""+ commiter[name][i]['login']+"""'
                    #"""
                    #nodes = graphDB_Session.run(cqlNodeQuery)


In [14]:
for x in issue_creator:
    #print("List of assignee of issues of the repos "+ x +": ", issue_creator[x])
    for i in range(len(issue_creator[x])):
        user = graph.nodes.match("u:user", name=issue_creator[x][i]['assignee']).first()
        repos = graph.nodes.match("r:repository", name=x).first()
        
        if type(repos) is not type(None):
            if type(user) is not type(None):
                #print(type(user) is not type(None))
                #print(type(user) is not type(None))
                ur = Relationship(user, ("a:assigned_issue"), repos)
                merge=graph.merge(ur)
                #print(repos,issue_creator[x][i]['assignee'] )
            else:
                user = Node("u:user", name=issue_creator[x][i]['assignee'])
                graph.create(user)
                user = graph.nodes.match("u:user", name=issue_creator[x][i]['assignee']).first()
                graph.merge(user, ('a:assigned_issue'), repos)
        #print(issue_creator[x][i]['assignee'], x, graph.nodes.match("r:repository", name=x).first())
        else:
            graph.create(ur)

In [ ]:
r = requests.get('https://api.github.com/user/mojombo/repos')
r.json()

In [63]:
for x in repos_stargazers_login:
    #print("List of assignee of issues of the repos "+ x +": ", issue_creator[x])
    for i in range(len(repos_stargazers_login[x])):
        user = graph.nodes.match("u:user", name=repos_stargazers_login[x][0][i]).first()
        repos = graph.nodes.match("r:repository", name=x).first()
        
        if type(repos) is not type(None):
            if type(user) is not type(None):
                #print(type(user) is not type(None))
                #print(type(user) is not type(None))
                ur = Relationship(user, ("s:starred"), repos, date=repos_stargazers_login[x][1][i])
                graph.merge(ur)
                #print(repos,issue_creator[x][i]['assignee'] )
            else:
                user = Node("u:user", name=repos_stargazers_login[x][0][i])
                graph.create(user)
                user = graph.nodes.match("u:user", name=repos_stargazers_login[x][0][i]).first()
                ur = Relationship(user, ("s:starred"), repos, date=repos_stargazers_login[x][1][i])
                graph.merge(ur)
        #print(issue_creator[x][i]['assignee'], x, graph.nodes.match("r:repository", name=x).first())
        else:
            graph.create(ur)

In [211]:

   
df = (graph.run("MATCH (user) RETURN user.name").to_table())
for i in range(len(df)):
    user = graph.nodes.match("u:user", name=df[i][0]).first()
    #print((df[i][0]))
    if type(df[i][0]) is not type(None):
        if len(graph.relationships.match((user, None), "s:starred")) !=0:
            #print(graph.relationships.match((user, None), "s:starred").__iter__())
            for obj in graph.relationships.match((user, None), "s:starred").__iter__():
                    print((obj))
#df

(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(calais)
(leahneukirchen)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gitsum)
(kevinclark)-[:s:starred

(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(calais)
(leahneukirchen)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gitsum)
(kevinclark)-[:s:starred

(topfunky)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(resource_controller)
(beccasaurus)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(markaby)
(eric)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(gchart)
(eric)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(duplikate)
(JGaudette)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(votigoto)
(apikeith)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(mofo)
(dan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(xhtmlize)
(joshua)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(bmhsearch)
(timothyjoh)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(simply_versioned)
(lukemelia)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gchart)
(lancecarlson)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(autozest)
(xwmx)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(switchpipe)
(msimkins)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(ebay4r)
(heavysixer)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(credit_card_tools)
(heavysixer)-[:s:starred {date: '2008-01-14T04:

(jnewland)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(audited)
(jnewland)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(capsize)
(jnewland)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(yaws)
(jnewland)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(calais)
(jnewland)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(xhtmlize)
(jnewland)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(votigoto)
(jnewland)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(lazy_record)
(jnewland)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(sinatra)
(jnewland)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(starling)
(ahmetabdi)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(css_naked_day)
(ahmetabdi)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(groomlake)
(ahmetabdi)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(schemr)
(ahmetabdi)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(mofo)
(ahmetabdi)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(phosphor)
(ahmetabdi)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(god)


(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(calais)
(leahneukirchen)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gitsum)
(kevinclark)-[:s:starred

...

(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(calais)
(leahneukirchen)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gitsum)
(kevinclark)-[:s:starred

(ezmobius)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(rubinius)
(benburkert)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(schemr)
(sr)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ape)
(sr)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(backpacking)
(sr)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(alogr)
(sr)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(merblogger)
(sr)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(dst)
(mojombo)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(glowstick)
(mojombo)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ambition)
(wycats)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(eycap)
(wycats)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rubinius)
(vanpelt)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(exception_logger)
(vanpelt)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(jsawesome)
(defunkt)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ambitious_activeldap)
(defunkt)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(cache_fu)
(defunkt)-[:s:starred {d

(mojombo)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(glowstick)
(mojombo)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ambition)
(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {dat

(drnic)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(jsawesome)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(autozest)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(alogr)
(wayneeseguin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(merblogger)
(peterc)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(switchpipe)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(credit_card_tools)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ram)
(up_the_irons)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(ebay4r)
(atmos)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(fitter_happier)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(uploadcolumn)
(jnicklas)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rorem)
(cristibalan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(awesomeness)
(cristibalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(calais)
(leahneukirchen)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(gitsum)
(kevinclark)-[:s:starred

(tweibley)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(forgery)
(ELLIOTTCABLE)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(soup)
(jeremy)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(rails)
(rmanalan)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(rails)
(gerrit)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(ape)
(ryanb)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(awesomeness)
(colin)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(acts_as_money)
(jordanfowler)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(acts_as_money)
(stan)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(clear_empty_attributes)
(smtlaissezfaire)-[:s:starred {date: '2008-01-14T04:33:35Z'}]->(clear_empty_attributes)
(Suave)-[:s:starred {date: '2008-01-13T10:19:47Z'}]->(css_naked_day)
